<a href="https://colab.research.google.com/github/Condemor-bit/Large-Language-Models-/blob/main/Q_A_PubMed_Advanced_RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a advanced RAG system using LlamaIndex and Zephyr-7b for PubMed searches. It is slower than simple, however is able to provide more relevant content

https://github.com/run-llama/llama-hub/blob/main/llama_hub/llama_packs/query/rag_fusion_pipeline/rag_fusion_pipeline.ipynb

11/01/2024

In [1]:
#@title 1º) Change the runtime environment to 'T4 GPU' and install the dependencies
#%%capture
!pip install -q --upgrade git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q llama-index
!pip install -q pypdf
!pip install -q docx2txt
!pip install -q llama_hub
#!pip install -q llama-index[local_models]
#!pip install -q llama-index[query_tools]
print("=========================")
print("Proceed to the next cell.")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

In [3]:
#@title 2º) Load the model

import warnings
warnings.simplefilter("ignore", UserWarning)

import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_hub.llama_packs.query.rag_fusion_pipeline.base import RAGFusionPipelinePack
from llama_index import download_loader



quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens= 1024,#256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)


#embed model

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5") #chunk_size=512, chunk_overlap=50)
set_global_service_context(service_context)



print("=========================")
print("Proceed to the next cell.")

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Proceed to the next cell.


#*At this point, you have the model and the dependencies installed. Avoid re-running these cells (1º and 2º).*




In [4]:
#@title 3º) Select the number of first articles and the keywords to query following the same format as for PubMed. Perform the search and create the query vectors

Max_results = 100 # @param {type:"integer"}
Search_Query = "(diabetes) AND (cardiovascular risk factors)"   #@param {type: "string"}

PubmedReader = download_loader("PubmedReader")
loader = PubmedReader()
docs = loader.load_data(search_query=f"""{Search_Query}""", max_results=Max_results)

pack = RAGFusionPipelinePack(docs, llm=llm)


https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776294&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776290&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776286&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776244&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776242&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776212&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776207&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776179&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776124&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776113&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776112&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776110&db=pmc
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?id=10776109&db=pmc

In [5]:
#@title 4º) Interact with the data
query = input("How can I help you?: ")
response = pack.run(query=f""" {query} """)
print(response)

How can I help you?: Explain me the relationship with diabetes and cardiovascular risk factors
Based on the provided information, there is a correlation between diabetes and cardiovascular risk factors. Studies have shown that aging patients with metabolic syndrome (MetS) in the higher-risk group with diabetes are significantly more likely to have higher systolic blood pressure, diastolic blood pressure, and triglyceride (TG) levels than those in the lower-risk group. Additionally, aging men with MetS and diabetes are significantly more likely to be clustered in the high-risk groups than women. However, lifestyle interventions targeting factors such as physical activity, healthy diet, and medication adherence have been associated with a reduction in cardiovascular risk factors, including total cholesterol and fasting glucose levels, in individuals with diabetes and higher cardiovascular disease (CVD) risk. Therefore, managing diabetes through lifestyle interventions may also help to re